In [ ]:
print("monopoly game with adverserial search")

monopoly game with adverserial search


In [ ]:
def minimax(state, depth, maximizing_player):
    if depth == 0 or state.is_terminal():
        return state.evaluate()

    if maximizing_player:
        best_value = float('-inf')
        for move in state.legal_moves():
            new_state = state.make_move(move)
            value = minimax(new_state, depth - 1, False)
            best_value = max(best_value, value)
        return best_value
    else:
        best_value = float('inf')
        for move in state.legal_moves():
            new_state = state.make_move(move)
            value = minimax(new_state, depth - 1, True)
            best_value = min(best_value, value)
        return best_value



#This is an implementation of the adverserial search algorithm, a decision-making algorithm used in game theory and artificial intelligence. The function takes in three parameters:

#state: The current state of the game, represented as an object or data structure.
#depth: The current depth of the search tree. The algorithm stops searching when it reaches a certain depth or when it reaches a terminal state (i.e., a state where the game is over).
#maximizing_player: A boolean value that indicates whether the current player is trying to maximize their score or minimize their opponent's score.
#The function returns the best score that the maximizing player can achieve given the current state and the remaining game tree.

#The function works by recursively exploring the game tree, starting from the current state. At each level of the tree, the function alternates between maximizing and minimizing the score. The maximizing player tries to choose a move that leads to the highest score, while the minimizing player tries to choose a move that leads to the lowest score. The function returns the best score found at the leaf nodes of the search tree, which represents the best move that the maximizing player can make.

In [ ]:
import random
# Define a class for a property space on the game board
class PropertySpace:
    def __init__(self, name, price, rent):
        self.name = name  # Name of the property
        self.price = price  # Price to purchase the property
        self.rent = rent  # Rent to be paid by other players when they land on the property
        self.owner = None  # The player who owns the property


# Define a class for a player
class Player:
    def __init__(self, name):
        self.name = name  # Player's name
        self.money = 1500  # Amount of money the player has at the start of the game
        self.position = 0  # Starting position of the player on the board
        self.properties = []  # Properties owned by the player
        self.num_jail_turns = 0  # Number of turns the player has spent in jail
        # self.has_monopoly = False  # Whether the player has a monopoly on a color group

    # Define a property for calculating the player's net worth

    @property
    def net_worth(self):
        return self.money + sum(p.price for p in self.properties)

    # Define a method to check if the player has a monopoly on a color group
    def has_monopoly(self, color):
      # Define a function to use as a predicate for filtering the player's properties
      def predicate(property_space):
            return isinstance(property_space,PropertySpace)  and property_space.owner == self

      # Return a boolean indicating whether the player has a monopoly on the color group
      return predicate



# Define a function for a player to buy a property
def buy_property(state):
    current_player = state.players[state.current_player]  # Get the current player
    current_space = state.board_layout[current_player.position]  # Get the space the player is currently on

    # If the space is a property and the player has enough money to buy it and no one else owns it, buy it
    if isinstance(current_space, PropertySpace) and current_space.price <= current_player.money and not current_space.owner:
        current_player.money -= current_space.price
        current_space.owner = current_player
        current_player.properties.append(current_space)

# Define a function to end the current player's turn
def end_turn(state):
    state.current_player = (state.current_player + 1) % len(state.players)  # Move to the next player
    state.turn_number += 1  # Increment the turn number

# Define a function to check if the game is over
def is_game_over(state):
    # Get a list of players who still have positive money
    remaining_players = [p for p in state.players if p.money >= 0]
    # Return a boolean indicating whether there is only one remaining player or the game has gone on for too long
    return len(remaining_players) == 1 or state.turn_number > 50


#This is a function named display_board that takes a state object as an argument and prints out the current state of the board,
#including the current turn number, the list of spaces on the board and their owner (if any) and price (if applicable),
#and the list of players and their current money and net worth.

def display_board(state):
    print("TURN", state.turn_number)
    for i, space in enumerate(state.board_layout):
        if isinstance(space, PropertySpace):
            if space.owner:
                owner = space.owner.name
            else:
                owner = "None"
            print(f"{i}: {space.name} ({owner}) - ${space.price}")
        else:
            print(f"{i}: {space}")

    print("PLAYERS")
    for player in state.players:
      #display player money and all net_worth
        print(player.name, player.money, player.net_worth)





In [ ]:
class MonopolyGameState:
    def __init__(self, board_layout, players, current_player, turn_number):
        #Initializes a new MonopolyGameState object with the given board layout, players, current player index, and turn number.
        self.board_layout = board_layout
        self.players = players
        self.current_player = current_player
        self.turn_number = turn_number

    def legal_moves(self):
        # Define the legal moves for the current game state
        moves = []

        current_player_index = self.current_player  # get index of current player
        print(current_player_index)
        # Add possible moves based on current player's position on the board
        current_player = self.players[current_player_index]
        print(current_player)
        current_space = self.board_layout[current_player.position]

        # Add move to roll dice
        moves.append("roll_dice")

        # Add move to buy property if player has enough money and space is available for purchase
        if isinstance(current_space, PropertySpace) and current_space.price <= current_player.money and not current_space.owner:
            moves.append("buy_property")

        # Add move to end turn
        moves.append("end_turn")

        return moves

    def evaluate(self):
        # Evaluate the current game state
        current_player = self.players[self.current_player]
        net_worth = current_player.net_worth

        # Add value for number of properties owned
        num_properties = len([s for s in self.board_layout if isinstance(s, PropertySpace) and s.owner == current_player])
        net_worth += num_properties * 5000

        # Subtract value for number of times player has been in jail
        net_worth -= current_player.num_jail_turns * 10000

        return net_worth

    def make_move(self, move):
        # Create a new game state by applying the given move to the current state
        new_board_layout = self.board_layout
        new_players = self.players.copy()
        new_current_player = self.current_player
        new_turn_number = self.turn_number + 1

        # Update the game state based on the move
        # Here is a hypothetical example:
        if move == "roll_dice":
            # Update the current player's position on the board
            current_player = self.players[self.current_player]
            current_player.position += random.randint(1, 6)
            current_player.position %= len(self.board_layout)
            new_players[self.current_player] = current_player

            # Check if the player has landed on a property space and can buy it
            current_space = self.board_layout[current_player.position]
            if isinstance(current_space,
                          PropertySpace) and current_space.price <= current_player.money and not current_space.owner:
                buy_property(self)

            # End the turn
            end_turn(self)

        elif move == "buy_property":
            buy_property(self)
            end_turn(self)

        elif move == "end_turn":
            end_turn(self)
        end_turn(self)

        # Return the new game state
        return MonopolyGameState(new_board_layout, new_players, new_current_player, new_turn_number)

    def is_terminal(self):
        # Check if the game is over
        for player in self.players:
            if player.money >= 1000000:
                return True
        return False



In [ ]:

def play_game():
    # Set up the game
    board_layout = [
        "GO",
        PropertySpace("Mediterranean Avenue", 60, 2),
        "Community Chest",
        PropertySpace("Baltic Avenue", 60, 4),
        "Income Tax",
        "Reading Railroad",
        PropertySpace("Oriental Avenue", 100, 6),
        "Chance",
        PropertySpace("Vermont Avenue", 100, 6),
        PropertySpace("Connecticut Avenue", 120, 8),
        "Jail",
        PropertySpace("St. Charles Place", 140, 10),
        "Electric Company",
        PropertySpace("States Avenue", 140, 10),
        PropertySpace("Virginia Avenue", 160, 12),
        "Pennsylvania Railroad",
        PropertySpace("St. James Place", 180, 14),
        "Community Chest",
        PropertySpace("Tennessee Avenue", 180, 14),
        PropertySpace("New York Avenue", 200, 16),
        "Free Parking"
    ]
    players = [Player("Player 1"), Player("Player 2")]
    current_player = 0
    turn_number = 1

    state = MonopolyGameState(board_layout, players, current_player, turn_number)

    x = 0
    while not is_game_over(state):
        x +=1
        # if x > 20:
        #     break
        print("\n" + "="*20)
        display_board(state)

        # Roll the dice
        dice_roll = random.randint(1, 6) + random.randint(1, 6)
        print(f"{state.players[state.current_player].name} rolled a {dice_roll}!")

        # Move the player
        state.players[state.current_player].position += dice_roll
        if state.players[state.current_player].position >= len(state.board_layout):
            state.players[state.current_player].position -= len(state.board_layout)
            state.players[state.current_player].money += 200
            print(f"{state.players[state.current_player].name} passed GO and collected $200!")

        # Take action based on the space the player landed on
        current_space = state.board_layout[state.players[state.current_player].position]
        if isinstance(current_space, PropertySpace):
            if not current_space.owner:
                # Ask the player if they want to buy the property
                print(f"{state.players[state.current_player].name}, do you want to buy {current_space.name} for ${current_space.price}?")
                answer = input("Enter y for yes or n for no: ")
                if answer.lower() == "y":
                    buy_property(state)
                else:
                    print(f"{state.players[state.current_player].name} did not buy {current_space.name}.")
            elif current_space.owner != state.players[state.current_player]:
                # Pay rent to the owner
                rent = current_space.rent
                if isinstance(current_space,
                          PropertySpace) and current_space.owner and current_space.owner != current_player:
                  rent *= 2
                print(f"{state.players[state.current_player].name} paid ${rent} in rent to {current_space.owner.name}.")
                state.players[state.current_player].money -= rent
                current_space.owner.money += rent
        elif current_space == "Community Chest":
            # Draw a community chest card
            print(f"{state.players[state.current_player].name} drew a Community Chest card.")
        elif current_space == "Chance":
            # Draw a chance card
            print(f"{state.players[state.current_player].name} drew a Chance card.")
        # Check if the player landed on jail
        if current_space == "Jail":
            print(f"{state.players[state.current_player].name} is just visiting jail")
        elif current_space == "Go To Jail":
            print(f"{state.players[state.current_player].name} goes to jail")
            state.players[state.current_player].position = 10
            state.players[state.current_player].num_jail_turns = 3

        # Check if the player is in jail
        if state.players[state.current_player].num_jail_turns > 0:
            state.players[state.current_player].num_jail_turns -= 1
            if state.players[state.current_player].num_jail_turns == 0:
                print(f"{state.players[state.current_player].name} is released from jail")

        # End the turn
        end_turn(state)

    # Game over
    remaining_players = [p for p in state.players if p.money >= 0]
    if len(remaining_players) == 1:
        #If there is only one player remaining with positive money, that player is declared the winner
        print(f"\n{remaining_players[0].name} wins!")
    else:
        #If all remaining players have the same amount of money, the game ends in a draw.
        print("\nThe game ends in a draw.")



In [ ]:
play_game()